# **Maestría en Inteligencia Artificial Aplicada**

## **Curso: Proyecto Integrador**

### Tecnológico de Monterrey

### Prof Dra. Grettel Barceló Alonso y Dr. Luis Eduardo Falcón Morales

## Avance IV de Proyecto

## Modelo Base

## Integrantes del Equipo:
### - Erika Cardona Rojas            A01749170
### - Miriam Bönsch                  A01330346
### - Mardonio Manuel Román Ramírez  A01795265

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import yaml

# Cargando Yaml
with open("../config.yaml", "r", encoding="utf-8") as file:
    config = yaml.safe_load(file)

# Importando modulos
import sys
from pathlib import Path

parent_folder = str(Path.cwd().parent)
if parent_folder not in sys.path:
    sys.path.append(parent_folder)

from src import models as m

In [4]:
# Cargando Base de Datos
df = pd.read_excel(r"../Data/DF_Pred_4.xlsx")

# MODELOS PARA LA VARIABLE **DELTA BDNF**

## Filtrado De Variables

Dada nuestra previa entrega, se utilizarán las variables más importantes en la RFECV

In [15]:
# Cargando las variables más importantes según RFECV 1
df_ranking = pd.read_excel('../Entregables/UdeBarcelona/RFECV.xlsx')

selected_features_xg = df_ranking.loc[df_ranking['Ranking'] == 1, 'Feature']

# Cargando las variables más importantes según RFECV 2
df_ranking = pd.read_excel('../Entregables/UdeBarcelona/RFECV_Lasso.xlsx')

selected_features_lasso = df_ranking.loc[df_ranking['Ranking'] == 1, 'Feature']

del df_ranking

In [ ]:
# Obteniendo variables únicas de ambos
unique_selected_features = list(set(pd.concat([selected_features_xg, selected_features_lasso])))

# Entrenamiento de Modelos

## Justificación de la Estrategia de Validación y Optimización de Modelos

Para la fase de modelado predictivo de este estudio, se ha implementado un flujo de trabajo de Machine Learning enfocado en maximizar la confiabilidad de las métricas de evaluación, mitigando los riesgos inherentes al trabajo con muestras de tamaño reducido. El código anterior se fundamenta en los siguientes pilares metodológicos:

### 1. Validación Cruzada Robusta (`RepeatedKFold`)

Dado el tamaño de la muestra, un K-Fold tradicional podría presentar una alta varianza en la estimación del error dependiendo de cómo se realicen las particiones aleatorias. En su lugar, se optó por una estrategia de **K-Fold Repetido** (`n_splits=7`, `n_repeats=5`). 
* Al usar 7 particiones, aseguramos que el conjunto de validación de cada pliegue contenga un número razonable de observaciones (aproximadamente 6), permitiendo que el conjunto de entrenamiento mantenga la mayor parte de la varianza original.
* Repetir el proceso 5 veces con semillas aleatorias diferentes estabiliza las métricas de evaluación (RMSE, MAE, R2), promediando el rendimiento sobre 35 escenarios de validación distintos. Esto ofrece una estimación del error de generalización mucho más sólida y menos susceptible al azar que un K-Fold simple o un Leave-One-Out (el cual puede sufrir de alta varianza frente a valores atípicos).

### 2. Prevención Estricta de Fuga de Datos (Data Leakage)
Para garantizar la validez de los resultados, es imperativo que las transformaciones aplicadas a los datos no filtren información del conjunto de validación al conjunto de entrenamiento. Para ello, se encapsuló el `StandardScaler` dentro de un objeto `Pipeline` de Scikit-Learn. De esta manera, durante cada una de las 35 iteraciones de la validación cruzada, el escalador se ajusta (*fit*) estrictamente sobre los pliegues de entrenamiento, y solo se utiliza para transformar el pliegue de validación. 

### 3. Optimización Estocástica de Hiperparámetros (`RandomizedSearchCV`)
En lugar de una búsqueda de cuadrícula exhaustiva (`GridSearchCV`), se implementó `RandomizedSearchCV`. Esta técnica evalúa una muestra aleatoria de combinaciones del espacio de hiperparámetros (limitado a `n_iter=40` o al máximo de combinaciones posibles). La literatura demuestra que la búsqueda aleatoria es computacionalmente más eficiente e igual de efectiva que la búsqueda en cuadrícula para encontrar modelos óptimos, evitando el desgaste de recursos en zonas del espacio de búsqueda con bajo impacto en la función de pérdida.

### 4. Arquitectura Reproducible y Escalable
El código se diseñó utilizando un patrón **Model Registry**, separando la definición lógica de los algoritmos de su ejecución. Junto con la integración de la librería `logging`, esta estructura asegura que los experimentos sean totalmente trazables, reproducibles y fácilmente extensibles en caso de que se requiera evaluar nuevos modelos o distribuciones de parámetros en futuras etapas de la investigación.

> ### Creación de DF X e Y

In [7]:
X_full = df[unique_selected_features]
y = df.loc[:,'delta_bdnf_Int']

# Separación en Train y Test (80/20)
X_train, X_test, y_train, y_test = train_test_split(X_full, y, test_size=0.2, random_state=42)

> ### ENTRENAMIENTO CON TODAS LAS VARIABLES IMPORTANTES
> #### SEGUN AMBAS TECNICAS DE RFECV

In [12]:
results = m.models_comparison_and_train(X_train, y_train, X_test, y_test, config['models_and_params'])

INFO:src.models:Optimizando modelo: Ridge
INFO:src.models:Ridge terminado en 2.44 segundos
INFO:src.models:Optimizando modelo: Lasso
INFO:src.models:Lasso terminado en 0.25 segundos
INFO:src.models:Optimizando modelo: ElasticNet
INFO:src.models:ElasticNet terminado en 0.60 segundos
INFO:src.models:Optimizando modelo: SVR
INFO:src.models:SVR terminado en 0.54 segundos
INFO:src.models:Optimizando modelo: KNN Regressor
INFO:src.models:KNN Regressor terminado en 1.77 segundos
INFO:src.models:Optimizando modelo: Decision Tree
INFO:src.models:Decision Tree terminado en 0.52 segundos
INFO:src.models:Optimizando modelo: MLP Regressor
INFO:src.models:MLP Regressor terminado en 45.28 segundos


In [13]:
# Generando DataFrame Ordenado
results_df_all = (
    pd.DataFrame(results)
      .sort_values("RMSE")
      .reset_index(drop=True)
)

results_df_all

,Modelo,RMSE,MAE,R2,Mejores Hiperparámetros,Tiempo (s)
0,Lasso,37.251315,24.177280,0.551005,"{'selection': 'random', 'alpha': 0.1}",0.25
1,SVR,45.624454,26.750614,0.326475,"{'kernel': 'linear', 'gamma': 'auto', 'epsilon...",0.54
2,MLP Regressor,50.656609,27.263717,0.169709,"{'solver': 'adam', 'hidden_layer_sizes': [100,...",45.28
3,KNN Regressor,52.128744,30.978001,0.120749,"{'weights': 'distance', 'p': 1, 'n_neighbors': 5}",1.77
4,ElasticNet,53.453129,28.581245,0.075505,"{'l1_ratio': 0.3, 'alpha': 1.0}",0.60
5,Ridge,56.825805,29.894210,-0.044839,"{'solver': 'saga', 'alpha': 100.0}",2.44
6,Decision Tree,78.411801,46.707459,-0.989396,"{'min_samples_split': 20, 'min_samples_leaf': ...",0.52


> ### ENTRENAMIENTO CON TODAS LAS VARIABLES IMPORTANTES
> #### SEGUN TECNICAS DE RFECV CON LASSO

In [19]:
results_l = m.models_comparison_and_train(X_train[selected_features_lasso], y_train, X_test[selected_features_lasso], y_test, config['models_and_params'])

INFO:src.models:Optimizando modelo: Ridge
INFO:src.models:Ridge terminado en 0.38 segundos
INFO:src.models:Optimizando modelo: Lasso
INFO:src.models:Lasso terminado en 0.21 segundos
INFO:src.models:Optimizando modelo: ElasticNet
INFO:src.models:ElasticNet terminado en 0.47 segundos
INFO:src.models:Optimizando modelo: SVR
INFO:src.models:SVR terminado en 1.43 segundos
INFO:src.models:Optimizando modelo: KNN Regressor
INFO:src.models:KNN Regressor terminado en 0.42 segundos
INFO:src.models:Optimizando modelo: Decision Tree
INFO:src.models:Decision Tree terminado en 0.50 segundos
INFO:src.models:Optimizando modelo: MLP Regressor
INFO:src.models:MLP Regressor terminado en 58.22 segundos


In [20]:
# Generando DataFrame Ordenado
results_df_l = (
    pd.DataFrame(results_l)
      .sort_values("RMSE")
      .reset_index(drop=True)
)

results_df_l

,Modelo,RMSE,MAE,R2,Mejores Hiperparámetros,Tiempo (s)
0,SVR,11.362285,8.561862,0.958228,"{'kernel': 'linear', 'gamma': 0.001, 'epsilon'...",1.43
1,Ridge,11.517473,9.105473,0.957079,"{'solver': 'sag', 'alpha': 1.0}",0.38
2,Lasso,13.170090,10.406601,0.943878,"{'selection': 'random', 'alpha': 0.1}",0.21
3,ElasticNet,16.572712,11.880565,0.911132,"{'l1_ratio': 0.5, 'alpha': 0.1}",0.47
4,MLP Regressor,36.597829,19.000866,0.566620,"{'solver': 'lbfgs', 'hidden_layer_sizes': [50]...",58.22
5,Decision Tree,53.006178,31.762609,0.090901,"{'min_samples_split': 20, 'min_samples_leaf': ...",0.50
6,KNN Regressor,53.580062,27.929991,0.071109,"{'weights': 'distance', 'p': 2, 'n_neighbors': 7}",0.42


> ### ENTRENAMIENTO CON TODAS LAS VARIABLES IMPORTANTES
> #### SEGUN TECNICAS DE RFECV CON XG-BOOST

In [21]:
results_xg = m.models_comparison_and_train(X_train[selected_features_xg], y_train, X_test[selected_features_xg], y_test, config['models_and_params'])

INFO:src.models:Optimizando modelo: Ridge
INFO:src.models:Ridge terminado en 0.41 segundos
INFO:src.models:Optimizando modelo: Lasso
INFO:src.models:Lasso terminado en 0.26 segundos
INFO:src.models:Optimizando modelo: ElasticNet
INFO:src.models:ElasticNet terminado en 0.58 segundos
INFO:src.models:Optimizando modelo: SVR
INFO:src.models:SVR terminado en 0.48 segundos
INFO:src.models:Optimizando modelo: KNN Regressor
INFO:src.models:KNN Regressor terminado en 0.36 segundos
INFO:src.models:Optimizando modelo: Decision Tree
INFO:src.models:Decision Tree terminado en 0.52 segundos
INFO:src.models:Optimizando modelo: MLP Regressor
INFO:src.models:MLP Regressor terminado en 50.67 segundos


In [22]:
# Generando DataFrame Ordenado
results_df_xg = (
    pd.DataFrame(results_xg)
      .sort_values("RMSE")
      .reset_index(drop=True)
)

results_df_xg

,Modelo,RMSE,MAE,R2,Mejores Hiperparámetros,Tiempo (s)
0,SVR,55.385941,33.555069,0.007439,"{'kernel': 'poly', 'gamma': 0.1, 'epsilon': 0....",0.48
1,MLP Regressor,55.879021,35.131110,-0.010313,"{'solver': 'adam', 'hidden_layer_sizes': [50, ...",50.67
2,KNN Regressor,55.908314,34.461050,-0.011372,"{'weights': 'distance', 'p': 1, 'n_neighbors': 5}",0.36
3,Ridge,56.338435,33.092817,-0.026994,"{'solver': 'saga', 'alpha': 100.0}",0.41
4,ElasticNet,56.533292,33.213172,-0.034110,"{'l1_ratio': 0.1, 'alpha': 10.0}",0.58
5,Lasso,58.085098,34.690302,-0.091661,"{'selection': 'cyclic', 'alpha': 10.0}",0.26
6,Decision Tree,78.411801,46.707459,-0.989396,"{'min_samples_split': 20, 'min_samples_leaf': ...",0.52


## CONCLUSION






---
# MODELOS PARA LA VARIABLE **BDNF**